In [1]:
import yaml
from app import *
from core.earthModel import *

Welcome to JupyROOT 6.30/02


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats
from scipy.stats import gaussian_kde

def calculate_weights(L):
    weights = np.exp(-0.5 * L)
    return weights / np.sum(weights)  # Normalize weights

def calculate_interval(data, weights, confidence, center=1):
    sorted_indices = np.argsort(np.abs(data - center))
    sorted_data = data[sorted_indices]
    sorted_weights = weights[sorted_indices]
    cumulative_weights = np.cumsum(sorted_weights)
    total_weight = cumulative_weights[-1]
    
    target_weight = confidence * total_weight
    
    def interpolate(target):
        idx = np.searchsorted(cumulative_weights, target)
        if idx == 0:
            return sorted_data[0]
        if idx == len(cumulative_weights):
            return sorted_data[-1]
        x0, x1 = sorted_data[idx-1], sorted_data[idx]
        y0, y1 = cumulative_weights[idx-1], cumulative_weights[idx]
        return x0 + (x1 - x0) * (target - y0) / (y1 - y0)
    
    interpolated_value = interpolate(target_weight)
    
    lower = center - np.abs(interpolated_value - center)
    upper = center + np.abs(interpolated_value - center)
    return lower, upper

def get_parameter_intervals(A, B, C, L, confidence=0.6827):
    weights = calculate_weights(L)
    interval_A = calculate_interval(A, weights, confidence)
    interval_B = calculate_interval(B, weights, confidence)
    interval_C = calculate_interval(C, weights, confidence)
    return interval_A, interval_B, interval_C

def plot_parameter_distribution(ax, data, weights, param_name, interval):
    kernel = gaussian_kde(data, weights=weights)
    x_range = np.linspace(data.min(), data.max(), 5000)
    y = kernel(x_range)
    
    ax.plot(x_range, y)
    ax.fill_between(x_range, y, alpha=0.3)
    ax.set_xlabel(param_name)
    ax.set_ylabel('Density')
    ax.set_title(f'{param_name} Distribution')
    
    interval_min, interval_max = interval
    ax.axvline(interval_min, color='r', linestyle='--')
    ax.axvline(interval_max, color='r', linestyle='--')
    ax.axvline(1, color='k', linestyle='-')  # Center line at 1
    ax.fill_between([interval_min, interval_max], 0, ax.get_ylim()[1], color='r', alpha=0.2)
    
    ax.text(0.05, 0.95, f'1σ Interval: [{interval_min:.2f}, {interval_max:.2f}]', 
            transform=ax.transAxes, verticalalignment='top')

def plot_2d_marginal(ax, x, y, weights, xlabel, ylabel):
    values = np.vstack([x, y])
    kernel = gaussian_kde(values, weights=weights)
    
    xx, yy = np.mgrid[x.min():x.max():100j, y.min():y.max():100j]
    positions = np.vstack([xx.ravel(), yy.ravel()])
    
    f = np.reshape(kernel(positions).T, xx.shape)
    
    ax.contourf(xx, yy, f, cmap='YlGnBu', levels=5)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(f'Marginal {xlabel}-{ylabel}')

def create_parameter_plots(A, B, C, L):
    weights = calculate_weights(L)
    interval_A, interval_B, interval_C = get_parameter_intervals(A, B, C, L)

    fig = plt.figure(figsize=(15, 10))
    
    ax_ab = fig.add_subplot(231)
    plot_2d_marginal(ax_ab, A, B, weights, 'A', 'B')
    ax_ac = fig.add_subplot(232)
    plot_2d_marginal(ax_ac, A, C, weights, 'A', 'C')
    ax_bc = fig.add_subplot(233)
    plot_2d_marginal(ax_bc, B, C, weights, 'B', 'C')
    
    ax_a = fig.add_subplot(234)
    plot_parameter_distribution(ax_a, A, weights, 'A', interval_A)
    ax_b = fig.add_subplot(235)
    plot_parameter_distribution(ax_b, B, weights, 'B', interval_B)
    ax_c = fig.add_subplot(236)
    plot_parameter_distribution(ax_c, C, weights, 'C', interval_C)
    
    plt.tight_layout()
    plt.show()

    return interval_A, interval_B, interval_C

In [3]:
config_file='/Users/cjesus/Documents/HKnuTomo/config/chi2_config.yaml'
with open(config_file,'r') as f:
    cfg=yaml.safe_load(f)

earth = EarthModel(cfg, None, None)

In [18]:
case = 'sin23_45'
master_dic = case+'/profile_LL_files/'

# tag_to_name = {
#     'Nominal': 'Nominal',
#     'FDS_A':  'Stat',
#     'FDS_B':  '0-10deg',
#     'FDS_C':  '0-20deg',
#     'FDS_D':  '0.3-0deg',
#     'FDS_E':  '0.5-0deg',
#     'FDS_F':  '0.3-10deg',
#     'FDS_G':  '0.5-20deg',
# }

tag_to_name = {
    'Nominal': 'Nominal',
    'FDS_A':  'Stat',
    'FDS_B':  '0-10deg',
    'FDS_D':  '30-0deg',
    #'FDS_E':  '0-20deg',
    #'FDS_G':  '0.5-30deg',
    'FDS_I':  '0.1-10deg',
    'FDS_H':  '0.2-10deg',
    'FDS_F':  '30-10deg',
    'FDS_C':  '0-20deg',
    'FDS_J':  '0.3-20deg',
}



tags = ['Nominal', 'FDS_A', 'FDS_B', 'FDS_D', 'FDS_I', 'FDS_H', 'FDS_F',  'FDS_C', 'FDS_J']
param_list = ['kappa_OC', 'kappa_IM', 'kappa_OM']

det_name = ['HK']


viability_ranges = {
    'IC': (0.7, 1.1),
    'OC': (0.7, 1.1),
    'IM': (0.85, 1.45),
    'OM': (0.35, 1.1)
}

In [21]:
def format_single_value(value, viability_range, is_upper):
    formatted = f"{value:.3f}"
    if (is_upper and value > viability_range[1]) or (not is_upper and value < viability_range[0]):
        #formatted = f"\\textcolor{{red}}{{{formatted}}}"
        formatted = "-"
    return formatted

reduced_results = {}
missing_names = []

for tag in tags:
    dfs = []
    for p_name in param_list:    
        fname = master_dic + 'HK_' + tag + '_' + p_name + '_1D.csv'
        if os.path.exists(fname):
            dfs.append(pd.read_csv(fname))
        else:
            missing_names.append(fname)
    if len(missing_names):
        continue
    
    A = np.array(list(dfs[0].OCk)+list(dfs[1].OCk)+list(dfs[2].OCk))
    B = np.array(list(dfs[0].IMk)+list(dfs[1].IMk)+list(dfs[2].IMk))
    C = np.array(list(dfs[0].OMk)+list(dfs[1].OMk)+list(dfs[2].OMk))
    L = np.array(list(dfs[0].chi2)+list(dfs[1].chi2)+list(dfs[2].chi2))
    
    inter_68 = get_parameter_intervals(A, B, C, L, 0.6827)
    inter_90 = get_parameter_intervals(A, B, C, L, 0.9)

    print(inter_68)
    break

    reduced_results[tag] = {
        'name': tag,
        'C_68_lower': format_single_value(inter_68[0][0], viability_ranges['OC'], False),
        'C_68_upper': format_single_value(inter_68[0][1], viability_ranges['OC'], True),
        'C_90_lower': format_single_value(inter_90[0][0], viability_ranges['OC'], False),
        'C_90_upper': format_single_value(inter_90[0][1], viability_ranges['OC'], True),
        'IM_68_lower': format_single_value(inter_68[1][0], viability_ranges['IM'], False),
        'IM_68_upper': format_single_value(inter_68[1][1], viability_ranges['IM'], True),
        'IM_90_lower': format_single_value(inter_90[1][0], viability_ranges['IM'], False),
        'IM_90_upper': format_single_value(inter_90[1][1], viability_ranges['IM'], True),
        'OM_68_lower': format_single_value(inter_68[2][0], viability_ranges['OM'], False),
        'OM_68_upper': format_single_value(inter_68[2][1], viability_ranges['OM'], True),
        'OM_90_lower': format_single_value(inter_90[2][0], viability_ranges['OM'], False),
        'OM_90_upper': format_single_value(inter_90[2][1], viability_ranges['OM'], True)
    }

# Dictionary to map the original names to the new names
name_mapping = {
    'Nominal': 'HK Nominal',
    'FDS_A': 'HK Stats.',
    'FDS_B': 'E$_{0\%}$  and 10$^\circ$',
    'FDS_C': 'E$_{0\%}$  and 20$^\circ$',
    'FDS_D': 'E$_{30\%}$ and 0$^\circ$',
    'FDS_E': 'E$_{50\%}$ and 0$^\circ$',
    'FDS_F': 'E$_{30\%}$ and 10$^\circ$',
    'FDS_G': 'E$_{50\%}$ and 20$^\circ$'
}

name_mapping = {
    'Nominal': 'HK Nominal',
    'FDS_A': 'HK Stats.',
    'FDS_B': 'E$_{0\%}$  \\& 10$^\circ$',
    'FDS_C': 'E$_{0\%}$  \\& 20$^\circ$',
    'FDS_D': 'E$_{30\%}$ \\& 0$^\circ$',
    #'FDS_E': 'E$_{50\%}$ \\& 0$^\circ$',
    'FDS_F': 'E$_{30\%}$ \\& 10$^\circ$',
    #'FDS_G': 'E$_{50\%}$ \\& 20$^\circ$',
    'FDS_H': 'E$_{20\%}$ \\& 10$^\circ$',
    'FDS_I': 'E$_{10\%}$ \\& 10$^\circ$',
    'FDS_J': 'E$_{30\%}$ \\& 20$^\circ$'
}



# Generate LaTeX table content for the comprehensive table
table_content = ""
for key, value in reduced_results.items():
    print(key)
    name = name_mapping.get(value['name'], value['name'])
    table_content += f"{name} & {value['C_68_lower']} & {value['C_68_upper']} & {value['C_90_lower']} & {value['C_90_upper']} & "
    table_content += f"{value['IM_68_lower']} & {value['IM_68_upper']} & {value['IM_90_lower']} & {value['IM_90_upper']} & "
    table_content += f"{value['OM_68_lower']} & {value['OM_68_upper']} & {value['OM_90_lower']} & {value['OM_90_upper']} \\\\\n\\hline\n"

# Add the SK Nominal row (you'll need to provide the actual values)
#sk_nominal = "SK Nominal & 0.85 & \\textcolor{red}{1.15} & 0.75 & \\textcolor{red}{1.25} & 0.85 & \\textcolor{red}{1.15} & 0.75 & \\textcolor{red}{1.25} & 0.80 & \\textcolor{red}{1.20} & \\textcolor{red}{0.65} & \\textcolor{red}{1.35}  \\\\\n\\hline\n"
sk_nominal = "SK Nominal& 0.854 & - & 0.750 & - & - & 1.205 & - & 1.350 & 0.698 & - & 0.500 & - \\\\\n\\hline\n"
table_content = sk_nominal + table_content

comprehensive_latex_table = f"""
\\begin{{table*}}[htbp]
\\centering
\\resizebox{{\\textwidth}}{{!}}{{
\\begin{{tabular}}{{|p{{2.2cm}}|C{{0.8cm}}|C{{0.8cm}}|C{{0.8cm}}|C{{0.8cm}}|C{{0.8cm}}|C{{0.8cm}}|C{{0.8cm}}|C{{0.8cm}}|C{{0.8cm}}|C{{0.8cm}}|C{{0.8cm}}|C{{0.8cm}}|}}
\\hline\\hline
 & \\multicolumn{{4}}{{c|}}{{Core (C)}} & \\multicolumn{{4}}{{c|}}{{Inner Mantle (IM)}} & \\multicolumn{{4}}{{c|}}{{Outer Mantle (OM)}} \\\\
\\hline
 & \\multicolumn{{2}}{{c|}}{{68\\%}} & \\multicolumn{{2}}{{c|}}{{90\\%}} & \\multicolumn{{2}}{{c|}}{{68\\%}} & \\multicolumn{{2}}{{c|}}{{90\\%}} & \\multicolumn{{2}}{{c|}}{{68\\%}} & \\multicolumn{{2}}{{c|}}{{90\\%}} \\\\
\\hline
Model & lower & upper & lower & upper & lower & upper & lower & upper & lower & upper & lower & upper \\\\
\\hline
{table_content}
\\hline\\hline
\\end{{tabular}}
}}
\\caption{{Lower and upper limits for Core (C), Inner Mantle (IM), and Outer Mantle (OM) expressed as factors of their nominal values corresponding to unity. Dashes indicate the limit is given by the Earth model conservations.}}
\\label{{tab:comprehensive_summary}}
\\end{{table*}}
"""

print(comprehensive_latex_table)
print('missing_names: ', missing_names)


\begin{table*}[htbp]
\centering
\resizebox{\textwidth}{!}{
\begin{tabular}{|p{2.2cm}|C{0.8cm}|C{0.8cm}|C{0.8cm}|C{0.8cm}|C{0.8cm}|C{0.8cm}|C{0.8cm}|C{0.8cm}|C{0.8cm}|C{0.8cm}|C{0.8cm}|C{0.8cm}|}
\hline\hline
 & \multicolumn{4}{c|}{Core (C)} & \multicolumn{4}{c|}{Inner Mantle (IM)} & \multicolumn{4}{c|}{Outer Mantle (OM)} \\
\hline
 & \multicolumn{2}{c|}{68\%} & \multicolumn{2}{c|}{90\%} & \multicolumn{2}{c|}{68\%} & \multicolumn{2}{c|}{90\%} & \multicolumn{2}{c|}{68\%} & \multicolumn{2}{c|}{90\%} \\
\hline
Model & lower & upper & lower & upper & lower & upper & lower & upper & lower & upper & lower & upper \\
\hline
SK Nominal& 0.854 & - & 0.750 & - & - & 1.205 & - & 1.350 & 0.698 & - & 0.500 & - \\
\hline

\hline\hline
\end{tabular}
}
\caption{Lower and upper limits for Core (C), Inner Mantle (IM), and Outer Mantle (OM) expressed as factors of their nominal values corresponding to unity. Dashes indicate the limit is given by the Earth model conservations.}
\label{tab:comprehensive_su

In [20]:
import numpy as np
import pandas as pd
import os

def format_single_value(value, viability_range, is_upper):
    formatted = f"{value:.2f}"
    if (is_upper and value > viability_range[1]) or (not is_upper and value < viability_range[0]):
        #formatted = f"\\textcolor{{red}}{{{formatted}}}"
        formatted = "-"
    return formatted

def get_single_parameter_interval(x, chi2, confidence):
    L = chi2 - np.min(chi2)
    weights = np.exp(-0.5 * L)
    weights /= np.sum(weights)
    
    sorted_indices = np.argsort(x)
    sorted_x = x[sorted_indices]
    sorted_weights = weights[sorted_indices]
    cumulative_weights = np.cumsum(sorted_weights)
    
    lower_target = (1 - confidence) / 2
    upper_target = 1 - (1 - confidence) / 2
    
    def interpolate(target):
        idx = np.searchsorted(cumulative_weights, target)
        if idx == 0:
            return sorted_x[0]
        if idx == len(cumulative_weights):
            return sorted_x[-1]
        x0, x1 = sorted_x[idx-1], sorted_x[idx]
        y0, y1 = cumulative_weights[idx-1], cumulative_weights[idx]
        return x0 + (x1 - x0) * (target - y0) / (y1 - y0)
    
    lower = interpolate(lower_target)
    upper = interpolate(upper_target)
    
    return lower, upper

param_list = ['OCR']
det_name = ['HK']

# You may need to adjust these ranges based on your data
viability_ranges = {
    'OCR': (0., 200000.0)  # Example range, adjust as needed
}

reduced_results = {}
for tag in tags:
    for det in det_name:
        dfs = []
        missing_names = []
        for p_name in param_list:    
            fname = master_dic + det + '_' + tag + '_' + p_name + '_1D.csv'
            print(fname)
            if os.path.exists(fname):
                df = pd.read_csv(fname)
                x = df['x'].values
                chi2 = df['chi2'].values
                
                inter_68 = get_single_parameter_interval(x, chi2, 0.6827)
                inter_90 = get_single_parameter_interval(x, chi2, 0.9)
                
                reduced_results[tag] = {
                    'name': tag,
                    'OCR_68_lower': format_single_value(inter_68[0], viability_ranges['OCR'], False),
                    'OCR_90_lower': format_single_value(inter_90[0], viability_ranges['OCR'], False),
                    'OCR_68_upper': format_single_value(inter_68[1], viability_ranges['OCR'], True),
                    'OCR_90_upper': format_single_value(inter_90[1], viability_ranges['OCR'], True)
                }
            else:
                missing_names.append(fname)
        print('missing files: ', missing_names)
        if len(missing_names):
            continue

def format_ocr_value(value):
    if float(value) == 2480:
        return '$<$2480'
    return f"{value}"


# Generate LaTeX table content for the reduced table
reduced_table_content = ""
for key, value in reduced_results.items():
    name = name_mapping.get(value['name'], value['name'])
    reduced_table_content += f"{name} & {format_ocr_value(value['OCR_68_lower'])} & {format_ocr_value(value['OCR_90_lower'])} & {format_ocr_value(value['OCR_68_upper'])} & {format_ocr_value(value['OCR_90_upper'])} \\\\\n\\hline\n"

# Add the SK Nominal row (you'll need to provide the actual values)
sk_nominal = "SK Nominal & 2716 & 2486 & 4127 & 4368  \\\\\n\\hline\n"
reduced_table_content = sk_nominal + reduced_table_content

# Generate the LaTeX table
reduced_latex_table = f"""
\\begin{{table}}[htbp]
\\centering
\\begin{{tabular}}{{|p{{2cm}}|C{{1.3cm}}|C{{1.3cm}}|C{{1.3cm}}|C{{1.3cm}}|}}
\\hline\\hline
  & \\multicolumn{{2}}{{c|}}{{lower OCR (km)}} & \\multicolumn{{2}}{{c|}}{{upper OCR (km)}} \\\\
\\hline
C.L. & 68\\% & 90\\% & 68\\% & 90\\% \\\\
\\hline\\hline
{reduced_table_content}
\\hline\\hline
\\end{{tabular}}
\\caption{{Reduced summary of results: lower and upper limits for OCR}}
\\label{{tab:reduced_summary_ocr}}
\\end{{table}}
"""

print(reduced_latex_table)

sin23_45/profile_LL_files/HK_Nominal_OCR_1D.csv
missing files:  ['sin23_45/profile_LL_files/HK_Nominal_OCR_1D.csv']
sin23_45/profile_LL_files/HK_FDS_A_OCR_1D.csv
missing files:  ['sin23_45/profile_LL_files/HK_FDS_A_OCR_1D.csv']
sin23_45/profile_LL_files/HK_FDS_B_OCR_1D.csv
missing files:  ['sin23_45/profile_LL_files/HK_FDS_B_OCR_1D.csv']
sin23_45/profile_LL_files/HK_FDS_D_OCR_1D.csv
missing files:  ['sin23_45/profile_LL_files/HK_FDS_D_OCR_1D.csv']
sin23_45/profile_LL_files/HK_FDS_I_OCR_1D.csv
missing files:  ['sin23_45/profile_LL_files/HK_FDS_I_OCR_1D.csv']
sin23_45/profile_LL_files/HK_FDS_H_OCR_1D.csv
missing files:  ['sin23_45/profile_LL_files/HK_FDS_H_OCR_1D.csv']
sin23_45/profile_LL_files/HK_FDS_F_OCR_1D.csv
missing files:  ['sin23_45/profile_LL_files/HK_FDS_F_OCR_1D.csv']
sin23_45/profile_LL_files/HK_FDS_C_OCR_1D.csv
missing files:  ['sin23_45/profile_LL_files/HK_FDS_C_OCR_1D.csv']
sin23_45/profile_LL_files/HK_FDS_J_OCR_1D.csv
missing files:  ['sin23_45/profile_LL_files/HK_FDS_J_O